# Unilever

### Automatización aplicación pagos

### Imports y configuración

In [88]:
import pandas as pd
import numpy as np

from datetime import datetime

from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import numbers

from openpyxl.styles import Alignment # Formatear columnas


### Remittance del Cliente

In [34]:
remittance = pd.read_excel(
    "Remittance.xlsx",
    skiprows=25,
    nrows=48,
    usecols=["DOC", "No. Doc", "Total a Pagar"]
).dropna(subset=["DOC"])

#### Transformaciones en Remittance

In [35]:
# Tranformo Remittence
## Cada cliente manda la informacion en Excel con distintos formatos (difrente indice) y a la vez para cada cliente la cantidad de filas va a variar

remittance = remittance.rename(columns={
    "DOC": "Tipo de Documento",
    "No. Doc": "Referencia / Factura",
    "Total a Pagar": "Importe de factura"
})

remittance["Referencia / Factura"] = remittance["Referencia / Factura"].str[1:-3]

remittance["Tipo de Documento"] = remittance["Tipo de Documento"].replace("Factura Comercial", "Factura")
remittance["Tipo de Documento"] = remittance["Tipo de Documento"].replace("Nota Crédito", "Descuentos no asociados a FC")

remittance["Importe de factura"] = pd.to_numeric(remittance["Importe de factura"], errors="coerce").round(2)

In [36]:
# Creamos columnas vacías (si no existen)
if "Descuento" not in remittance.columns:
    remittance["Descuento"] = ""

if "Motivo del descuento" not in remittance.columns:
    remittance["Motivo del descuento"] = ""

# Definimos condiciones y valores
conds = [
    (remittance["Referencia / Factura"].str.startswith("PMP", na=False)) & (remittance["Importe de factura"] < 0),
    remittance["Referencia / Factura"].str.startswith("0085489", na=False),
    remittance["Referencia / Factura"].str.startswith("463", na=False),
    remittance["Referencia / Factura"].str.startswith("9801649", na=False)
]

descuentos = ["RECHAZO", "DESCUENTO", "AVERIA", "FACT PROVEEDOR"]
motivos = ["551", "987", "522", "CSB"]

# Asignamos Descuento y Motivo del descuento
remittance["Descuento"] = np.select(conds, descuentos, default=remittance["Descuento"])
remittance["Motivo del descuento"] = np.select(conds, motivos, default=remittance["Motivo del descuento"])


### Cartera FBL5N del Cliente

In [37]:
FBL5N = pd.read_excel(
    "FBL5N Olimpica Ago 18.xlsx",
    sheet_name="FBL5N Olimpica Ago 18",
    usecols=["Type", "Reference", "    Amt in loc.cur."]
)
# Agregar a la importacion de cartera FBL5N que traiga info cuando: Type = "YE" y RCd = "NRO"
# Filtrar directamente Type == RV
FBL5N = FBL5N[FBL5N["Type"] == "RV"]

### Transformaciones en Cartera FBL5N

In [38]:
# Renombrar columnas
FBL5N = FBL5N.rename(columns={
    "Reference": "Referencia / Factura",
    "    Amt in loc.cur.": "importe_FBL5N"
}).reset_index(drop=True)

In [39]:
# Tipifico correctamente dato importe_FBL5N
# Paso 1: eliminar puntos de miles y reemplazar coma por punto decimal
FBL5N["importe_FBL5N"] = FBL5N["importe_FBL5N"].str.replace(".", "", regex=False)  # quita separador de miles
FBL5N["importe_FBL5N"] = FBL5N["importe_FBL5N"].str.replace(",", ".", regex=False)  # convierte decimal a punto
# Paso 2: convertir a float
FBL5N["importe_FBL5N"] = pd.to_numeric(FBL5N["importe_FBL5N"], errors="coerce")

### Merge de Remittance y FBL5N por Referencia / Factura (hrc_template)

In [40]:
# Cruzo Remittance y FBL5N por "Referencia / Factura"
hrc_template = pd.merge(
    remittance,
    FBL5N,
    on="Referencia / Factura",
    how="left" # mantiene todas las filas de remittance
)


#### Transformaciones en hrc_template

In [41]:
# Inicializamos la columna como NaN
hrc_template["Diferencia"] = pd.NA

# Calculamos la diferencia solo para facturas
hrc_template.loc[hrc_template["Tipo de Documento"] == "Factura", "Diferencia"] = (
     hrc_template["importe_FBL5N"] - hrc_template["Importe de factura"]
)

In [42]:
# Filtramos filas donde Diferencia no es NA y distinta de 0 (esto cambia con respecto al Template, menos dos registros)
diferencias = hrc_template[hrc_template["Diferencia"].notna() & (hrc_template["Diferencia"] != 0)].copy()

# Creamos las nuevas filas según tus reglas
registros_diferencias_entre_remittence_cartera = pd.DataFrame({
    "Tipo de Documento": "Descuentos no asociados a FC",
    "Referencia / Factura": diferencias["Referencia / Factura"],
    "Importe de factura": diferencias["Diferencia"],
    "Pago Neto": "",  # opcional
    "Descuento": diferencias["Diferencia"].apply(
        lambda x: "MENORES VALORES" if -2000 < x < 2000 else "A definir"
    ),
    "Motivo del descuento": diferencias["Diferencia"].apply(
        lambda x: "WOB" if x < 0 else "384"
    )
})

# Concatenamos las nuevas filas al DataFrame original
hrc_template = pd.concat([hrc_template, registros_diferencias_entre_remittence_cartera], ignore_index=True)


In [43]:
# Agrego dato Comentario
hrc_template["Comentarios"] = np.where(
    hrc_template["Tipo de Documento"] != "Factura",
    hrc_template["Descuento"].fillna("") + " " + hrc_template["Referencia / Factura"].fillna(""),
    ""
)
# Agrego dato Pago Neto
hrc_template["Pago Neto"] = hrc_template["Importe de factura"]

In [44]:
# Limpio las columnas con las que me voy a quedar en Template ordenadas

columnas_finales = [
    "Tipo de Documento",
    "Referencia / Factura",
    "Importe de factura",
    "Descuento",
    "Motivo del descuento",
    "Pago Neto",
    "Comentarios"
]

hrc_template = hrc_template[columnas_finales]

### Creacion de inputs para cuadros en Template

##### Dato Referencia de pago

In [59]:
# --- Paso 1: Leer la celda C10 de Remittance.xlsx ---
wb_rem = load_workbook("Remittance.xlsx", data_only=True)
ws_rem = wb_rem.active  # o ws_rem = wb_rem['NombreHoja'] si conoces la hoja
celda = ws_rem["C10"].value  # Ej: "Orden de Pago: PK003702815749"

In [69]:
# --- Paso 2: Extraer solo el número después de "Orden de Pago: " ---
if celda is not None and "Orden de Pago:" in celda:
    numero_orden = celda.split("Orden de Pago:")[1].strip()
else:
    numero_orden = ""

##### Dato del cliente

In [79]:
# Solo leemos la primera fila y las columnas necesarias
fbl5n = pd.read_excel(
    "FBL5N Olimpica Ago 18.xlsx",
    usecols=["Customer", "Name 1"],
    nrows=1
)

# Extraemos los valores
id_cliente = fbl5n["Customer"].iloc[0]
nombre_cliente = fbl5n["Name 1"].iloc[0]

##### Datos del pago

In [90]:
# Fecha
# Abrimos el archivo FBL3N
wb_fbl3n = load_workbook("FBL3N.xlsx", data_only=True)
ws_fbl3n = wb_fbl3n.active
# Leemos la celda K8 (fecha en formato 'dd.mm.yyyy')
fecha_original = ws_fbl3n["K8"].value  # ejemplo: '11.08.2025'
# Convertimos a objeto datetime
fecha_dt = datetime.strptime(fecha_original, "%d.%m.%Y")
# Formateamos en 'm/d/yy'
fecha_pago = fecha_dt.strftime("%-m/%-d/%y")  # en Windows puede que uses "%#m/%#d/%y

In [126]:
# Datos de montos
# Calculamos la suma de la columna "Pago Neto"
total_pago_neto = hrc_template["Pago Neto"].sum()
# Calculamos la suma de la columna "Pago Neto" en MENORES VALORES
menores_valores_sum = hrc_template.loc[hrc_template["Descuento"] == "MENORES VALORES", "Pago Neto"].sum()
# Sacamos diferencia
total_pago_real = total_pago_neto - menores_valores_sum

### Exportacion de archivo (Template_HRC)

##### Configuracion

In [136]:
ruta_salida = "Template_HRC.xlsx"

# Exportamos con pandas, indicando hoja y posición inicial
hrc_template.to_excel(
    ruta_salida,
    index=False,
    sheet_name="Template",
    startrow=17,
    startcol=2
)

In [137]:
# Abrimos el archivo para aplicar formatos y cuadros
wb = load_workbook(ruta_salida)
ws = wb["Template"]

### Generacion de cuadros

In [138]:
# Titulos Template
ws["C2"] = "Desglose de Pago"
ws["C4"] = "CAMPOS NO EDITABLES"

In [139]:
# Cuadro REFERENCIA DE PAGO
ws["G2"] = "REFERENCIA DE PAGO"
ws["H2"] = numero_orden # --- Dato dinámico ---

In [140]:
# Cuadro Informacion clinete
ws["C6"] = "Cliente" 
ws["C8"] = "Codigo de Cliente" 
ws["D6"] = nombre_cliente # --- Dato dinámico ---
ws["D8"] = id_cliente # --- Dato dinámico ---


In [141]:
# Cuadro Datos del pago
ws["C12"] = "Referencia"
ws["C13"] = numero_orden # --- Dato dinámico ---
ws["D12"] = "Fecha"
ws["D13"] = fecha_pago # --- Dato dinámico ---
ws["E12"] = "Método de Pago"
ws["E13"] = "Transferencia"
ws["F12"] = "Valor"
ws["F13"] = total_pago_real # --- Dato dinámico ---


In [142]:
# Cuadro de montos
ws["F6"] = "TOTAL s/ BANCOS"
ws["G6"] = total_pago_real # --- Dato dinámico ---
ws["F7"] = "TOTAL s/ DETALLE"
ws["G7"] = total_pago_neto # --- Dato dinámico ---
ws["F8"] = "DIFERENCIA"
ws["G8"] = -menores_valores_sum # --- Dato dinámico ---

In [143]:
# Formato del Template:

# Datos numericos en cuadros
for cell in ["G6", "G7", "G8", "F13"]:
    ws[cell].number_format = '#,##0.00'

# Formato Tabla principal
# Columnas numéricas
num_cols = ["Importe de factura", "Pago Neto"]

for col in num_cols:
    col_idx = hrc_template.columns.get_loc(col) + 3  # startcol=2 → columna C = 3 en openpyxl
    for row in range(18, 18 + len(hrc_template) + 1):  # largo de df +1
        ws.cell(row=row, column=col_idx).number_format = '#,##0.00'
        
# Columnas de texto
## Columnas de texto formateadas y centradas (excepto 'Comentarios')
str_cols = ["Tipo de Documento", "Referencia / Factura", "Descuento", "Motivo del descuento", "Comentarios"]

for col in str_cols:
    col_idx = hrc_template.columns.get_loc(col) + 3
    for row in range(18, 18 + len(hrc_template) + 1):  # largo de df +1
        cell = ws.cell(row=row, column=col_idx)
        cell.number_format = '@'  # formato texto
        if col != "Comentarios":
            cell.alignment = Alignment(horizontal="center", vertical="center")

In [144]:
# Guardar cambios en el archivo Excel
wb.save(ruta_salida)
print(f"Archivo exportado correctamente con formato: {ruta_salida}")

Archivo exportado correctamente con formato: Template_HRC.xlsx
